#  Groq API Assignment

This notebook contains implementation for:
- **Task 1:** Manage conversation history and periodic summarization
- **Task 2:** Extract structured info from chats using OpenAI-compatible Groq API function calling

---

In [16]:

!pip install --upgrade openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1


In [16]:
from openai import OpenAI
import json

# Initialize Groq-compatible OpenAI client
client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)

# Use two different models
MODEL_CHAT = "llama-3.1-8b-instant"
MODEL_FUNCTION = "llama-3.3-70b-versatile"



In [12]:
#Task-1
conversation = []
conversation_summary = []
summary_interval = 3
summary_counter = 0

def add_message(role, content):
    conversation.append({"role": role, "content": content})

def truncate_by_turns(history, n):
    return history[-n:]

def truncate_by_characters(history, max_chars):
    truncated, total_chars = [], 0
    for msg in reversed(history):
        total_chars += len(msg['content'])
        if total_chars > max_chars:
            break
        truncated.insert(0, msg)
    return truncated

def summarize(history):
    messages = [{"role": "system", "content": "Summarize the following chat briefly."}] + history
    response = client.chat.completions.create(
        model=MODEL_CHAT,
        messages=messages
    )
    return response.choices[0].message.content

def update_conversation(role, content):
    global summary_counter
    add_message(role, content)
    summary_counter += 1
    if summary_counter >= summary_interval:
        summary = summarize(conversation)
        conversation_summary.append({"summary": summary})
        summary_counter = 0


In [13]:
# Test Task 1
update_conversation("user", "Hi, I’m Devroop. I live in Bangalore.")
update_conversation("assistant", "Nice to meet you, Devroop! How can I help you?")
update_conversation("user", "I’m 21 years old. You can email me at dasguptadevroop@gmail.com or call 9876543210.")
update_conversation("assistant", "Thanks! Would you like to know about our services?")
update_conversation("user", "Yes, tell me about premium plans.")
update_conversation("assistant", "Premium includes all features plus 24/7 support.")

print(" Full Conversation:")
print(json.dumps(conversation, indent=2))
print("\n Truncated (last 4 messages):")
print(json.dumps(truncate_by_turns(conversation, 4), indent=2))
print("\n Truncated (max 150 characters):")
print(json.dumps(truncate_by_characters(conversation, 150), indent=2))
print("\n Summaries:")
print(json.dumps(conversation_summary, indent=2))



 Full Conversation:
[
  {
    "role": "user",
    "content": "Hi, I\u2019m Devroop. I live in Bangalore."
  },
  {
    "role": "assistant",
    "content": "Nice to meet you, Devroop! How can I help you?"
  },
  {
    "role": "user",
    "content": "I\u2019m 21 years old. You can email me at dasguptadevroop@gmail.com or call 9876543210."
  },
  {
    "role": "assistant",
    "content": "Thanks! Would you like to know about our services?"
  },
  {
    "role": "user",
    "content": "Yes, tell me about premium plans."
  },
  {
    "role": "assistant",
    "content": "Premium includes all features plus 24/7 support."
  }
]

 Truncated (last 4 messages):
[
  {
    "role": "user",
    "content": "I\u2019m 21 years old. You can email me at dasguptadevroop@gmail.com or call 9876543210."
  },
  {
    "role": "assistant",
    "content": "Thanks! Would you like to know about our services?"
  },
  {
    "role": "user",
    "content": "Yes, tell me about premium plans."
  },
  {
    "role": "assist

In [14]:
function_schema = {
    "name": "extract_user_info",
    "description": "Extracts user details from chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

def extract_info(chat):
    response = client.chat.completions.create(
        model=MODEL_FUNCTION,
        messages=[{"role": "user", "content": chat}],
        tools=[{"type": "function", "function": function_schema}],
        tool_choice="auto"
    )
    # function calling output
    return json.loads(response.choices[0].message.tool_calls[0].function.arguments)

In [17]:
samples = [
    "Hi, I’m Devroop. I live in Bangalore. I’m 21 years old. Email: dasguptadevroop@gmail.com. Phone: 9876543210.",
    "Hey! I’m Rhea from Mumbai, 25 years old. Email me at rhea123@gmail.com. Phone is 8765432109.",
    "This is Ajay. I'm 30, living in Delhi. Email: ajay@xyz.com, Call me at 7654321987."
]

for chat in samples:
    print("\n Extracted Info:")
    print(extract_info(chat))



 Extracted Info:
{'age': 21, 'email': 'dasguptadevroop@gmail.com', 'location': 'Bangalore', 'name': 'Devroop', 'phone': '9876543210'}

 Extracted Info:
{'age': 25, 'email': 'rhea123@gmail.com', 'location': 'Mumbai', 'name': 'Rhea', 'phone': '8765432109'}

 Extracted Info:
{'age': 30, 'email': 'ajay@xyz.com', 'location': 'Delhi', 'name': 'Ajay', 'phone': '7654321987'}
